### 投開票所轉換教學
<p>
此Python腳本示範如何將中選會投開票所的公開資訊轉換成選舉模組需要使用的對照表，在執行該檔案的腳本之前，請先利用election_hisotry中的
<a href="https://github.com/readr-data/election_history/blob/master/liCode_voter.Rmd">liCode_voter.Rmd</a>
腳本將網路上的投開票所資訊爬蟲到該資料夾底下，並設定cec_tbox_filepath為該檔案路徑。完成後再執行下面的函式產生對照表。
</p>

### 安裝並引入相依套件

In [14]:
!pip install --upgrade pip
!pip install -r requirements.txt

  Using cached pandas-2.2.0-cp39-cp39-win_amd64.whl.metadata (19 kB)
Using cached pandas-2.2.0-cp39-cp39-win_amd64.whl (11.6 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
  Rolling back uninstall of pandas
  Moving to c:\users\user\anaconda3\lib\site-packages\pandas-2.1.4.dist-info\
   from C:\Users\User\anaconda3\Lib\site-packages\~andas-2.1.4.dist-info
  Moving to c:\users\user\anaconda3\lib\site-packages\pandas\__init__.py
   from C:\Users\User\AppData\Local\Temp\pip-uninstall-u4boahxy\__init__.py
  Moving to c:\users\user\anaconda3\lib\site-packages\pandas\__pycache__\
   from C:\Users\User\anaconda3\Lib\site-packages\pandas\~_pycache__
  Moving to c:\users\user\anaconda3\lib\site-packages\pandas\_config\
   from C:\Users\User\anaconda3\Lib\site-packages\pandas\~config
  Moving to c:\users\user\anaconda3\lib\site-packages\pandas\_libs\
   from C:\Users\User\anaconda3\Lib\si

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\anaconda3\\Lib\\site-packages\\pandas.libs\\msvcp140-fa0758dedafbbe194d3ee96e3dc2b9a3.dll'
Consider using the `--user` option or check the permissions.



In [15]:
import json
import os
import copy
import pandas as pd
from collections import defaultdict

### 系統操作函式

In [16]:
def save_file(destination_file, data):
    dirname = os.path.dirname(destination_file)
    if len(dirname)>0 and not os.path.exists(dirname):
        os.makedirs(dirname)
    with open(destination_file, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
        print(f'Save data to {destination_file} successed.')

### 產生mapping表

In [17]:
def gen_tbox_mapping(filepath: str):
    result = {}
    tbox_info = pd.read_csv(filepath)
    # tbox_info.columns = ['city', 'tboxNo', 'tboxName', 'address']
    tbox_info.columns = ['tboxNo', 'village', 'countyCode', 'townCode']
    tbox_info['village'] = tbox_info['village'].str.replace('\n','')
    tbox_info['countyCode'] = tbox_info['countyCode'].astype(str).apply(lambda x: x.zfill(5))
    tbox_info['townCode'] = tbox_info['townCode'].astype(str).apply(lambda x: x.zfill(3))
    tbox_info['tboxNo'] = tbox_info['tboxNo'].astype(str)

    for i in range(len(tbox_info)):
        data = tbox_info.iloc[i]
        countyCode = data['countyCode']
        townCode   = data['townCode']
        tboxNo     = data['tboxNo']
        village    = data['village']

        subCounty = result.setdefault(countyCode, {})
        subTown = subCounty.setdefault(townCode, {})
        subTbox = subTown.setdefault(tboxNo, [])
        subTbox.append(village)
    return result

In [18]:
cec_tbox_filepath = os.path.join('.', 'cec-data', '20240113_tbox.csv') ### TODO: 需要使用爬蟲重新產生[time]_tbox.csv檔案
mapping_tbox = gen_tbox_mapping(cec_tbox_filepath)
mapping_tbox

{'63000': {'010': {'597': ['莊敬里'],
   '598': ['莊敬里'],
   '599': ['莊敬里'],
   '600': ['莊敬里'],
   '601': ['東榮里'],
   '602': ['東榮里'],
   '603': ['東榮里'],
   '604': ['東榮里'],
   '605': ['東榮里'],
   '606': ['三民里'],
   '607': ['三民里'],
   '608': ['三民里'],
   '609': ['三民里'],
   '610': ['三民里'],
   '611': ['新益里'],
   '612': ['新益里'],
   '613': ['新益里'],
   '614': ['富錦里'],
   '615': ['富錦里'],
   '616': ['富錦里'],
   '617': ['富錦里'],
   '618': ['新東里'],
   '619': ['新東里'],
   '620': ['新東里'],
   '621': ['新東里'],
   '622': ['富泰里'],
   '623': ['富泰里'],
   '624': ['富泰里'],
   '625': ['介壽里'],
   '626': ['介壽里'],
   '627': ['介壽里'],
   '628': ['精忠里'],
   '629': ['精忠里'],
   '630': ['精忠里'],
   '631': ['東光里'],
   '632': ['東光里'],
   '633': ['東光里'],
   '634': ['東光里'],
   '635': ['東光里'],
   '636': ['東光里'],
   '637': ['龍田里'],
   '638': ['龍田里'],
   '639': ['龍田里'],
   '640': ['龍田里'],
   '641': ['龍田里'],
   '642': ['龍田里'],
   '643': ['龍田里'],
   '644': ['東昌里'],
   '645': ['東昌里'],
   '646': ['東勢里'],
   '647': ['東勢里'],
   '648': ['東勢里

在產生tbox的對照表後，請將其存檔並手動放入選舉模板中的mapping資料夾底下